In [ ]:
import os
from dotenv import load_dotenv
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load the data
df_locations = pd.read_csv('./locations.csv')
df_hotels = pd.read_csv('./hotels.csv')

In [8]:
import openai
# Set the API key
openai.api_key = OPENAI_API_KEY
# create a function to generate the text

def generate_summary(name, text):
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": "You are an Imputer for a dataframe. This dataframe contains the name of a location in Victoria, BC Canada multiple reviews of the location. Each review is separated by three newline characters. \n Your job is to use what you know about the location as well as the reviews to come up with a summary of the location. \n"},
            {"role": "user", "content": f"name: {name}\n reviews: {text}"},
        ],
    frequency_penalty=0.8,
    temperature=0.3,
    max_tokens=500,
    top_p=0.8,
    )
    return response.choices[0].message.content


In [10]:
name = df_hotels.iloc[0]['name']
text = df_hotels.iloc[0]['reviews']

summary = generate_summary(name, text)

In [11]:
# write the summary text to a file
with open('summary.txt', 'w') as f:
    f.write(summary)